Find cross section of actors and movies?


In [ ]:
// TODO: use papa parse to load tsv files from imdb
var fs = require('fs');
var papa = require('papaparse');
var sqlite3 = require('better-sqlite3');

var principals = '/Users/briancullinan/Downloads/title.principals.tsv';
var movies = './movies.sqlite.db';

var db = new sqlite3(movies, {});

function importSQL(file, cb) {
    if(typeof file === 'string') {
        file = fs.createReadStream(file, { highWaterMark: 1024 * 1024 });
    }
    var prev = 0;
    var count = 0;
    return new Promise(resolve => papa.parse(file, {
        worker: true,
        delimiter: '\t',
        fastMode: true,
        step: function(results) {
            count++;
            if(count > prev + 1000) {
                prev = count;
                console.log(`${count} - ${results.data}`);
            }
            return cb(results.data);
        },
        complete: resolve
    }));
}

function createTables() {
    db.prepare(`
CREATE TABLE IF NOT EXISTS principals (
    tconst TEXT NOT NULL,
    ordering INT NOT NULL,
    nconst TEXT NOT NULL,
    category TEXT,
    job TEXT,
    characters TEXT,
    UNIQUE(tconst,ordering),
    PRIMARY KEY (tconst, ordering)
);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_titles
ON principals (tconst);`).run()
    db.prepare(`
CREATE INDEX IF NOT EXISTS principles_names
ON principals (nconst);`).run()
    return Promise.resolve();
}

function insertPrincipals(data) {
    return db.prepare(`
INSERT OR IGNORE INTO principals
    (tconst, ordering, nconst, category, job, characters)
VALUES (?,?,?,?,?,?)`).run(data);
}

$$.async();
createTables()
    .then(() => importSQL(principals, insertPrincipals))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
    // close the database connection
    .then(() => db.close())



/*
params = [[1,2],[3,4],[5,6],[7,8]]

stmt = db.prepare("INSERT INTO test (num1, num2) VALUES (?1,?2)");
stmt.all([params], function(err, res) {
    ...
});
*/



In [2]:
var sqlite3 = require('better-sqlite3');
var movies = './movies.sqlite.db';
var db = new sqlite3(movies, {});

function select(stmt, data) {
    return db.prepare(stmt).get(data);
}

function getTitles(t) {
    return select(`SELECT DISTINCT tconst FROM principals WHERE tconst=?`, typeof t === 'string' ? [t] : t);
}

function getTitlesByName(n) {
    return select(`SELECT DISTINCT tconst FROM principals WHERE nconst=?`, typeof n === 'string' ? [n] : n);
}

$$.async();
getTitlesByName('nm0001348')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
    // close the database connection
    .then(() => db.close())


RangeError: Too many parameter values were provided